# Membuat rangkuman otomatis menggunakan machine learning

## Sumber dari berbagai situs dan video

### Install & Import
Note: Sebelumnya, silahkan install [pyTorch](https://pytorch.org/) terlebih dahulu sebelum memasang library di bawah ini.

In [1]:
!pip install transformers

In [2]:
# Import class requests
import requests
# Import class pipeline dari library transformers
from transformers import pipeline
# Import class BeautifulSoup dari library bs4
from bs4 import BeautifulSoup

### Load Pipline
Note: Sebelum memulai, load summarization akan mengunduh model sshleifer/distilbart-cnn-12-6 sekitar 1 GB. Persiapkan bandwidth terlebih dahulu. Jangan khawatir, download hanya di lakukan sekali.

In [3]:
# Load pipeline dan mengunduh model sshleifer/distilbart-cnn-12-6
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


### Mengambil data artikel dari website

In [4]:
URL = "https://simamaung.com/dipermalukan-persebaya-3-0-mimpi-buruk-pertahanan-persib/"

In [5]:
r = requests.get(URL)

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [7]:
ARTICLE

'Dipermalukan Persebaya 3-0, Mimpi Buruk Pertahanan Persib \nPertahanan Persib rapuh dihadapan Persebaya Surabaya dalam bentrok big match Liga 1 Rabu (8/12/2021) malam di Stadion Maguwoharjo Sleman. Maung Bandung tak berdaya di babak kedua, aksi Taisei Marukawa dan Bruno Soares mempermalukan pertahanan Persib lewat tiga gol yang bersarang. Persebaya memperlihatkan permainan efektif, meski di babak pertama didominasi Persib. Aji Santoso bisa membalikkan situasi permainan di babak kedua dengan pergantian kunci memasukkan Rizky Ridho. Intensitas serangan ditingkatkan rencananya berbuah manis lewat gelontoran gol yang memukau. Babak I\nPersib memainkan kembali formasi 4-3-3, Dedi Kusnandar, Marc Klok, dan Beckham jadi pilihan utama sebagai motor permainan. Penguasaan bola diambil alih sejak menit pertama, lini tengah Persib begitu dominan dari pada Persebaya, dengan variasi umpan satu dua sentuhan sesekali umpan panjang. Frets Butuan melepas tendangan mengena tiang menit 4. Beckham Putra m

### Memotong text menjadi beberapa bagian (Chunk)

In [8]:
max_chunk = 250

In [9]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [10]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [11]:
len(chunks)

3

### Membuat kesimpulan

In [12]:
res = summarizer(chunks, max_length=200, min_length=30, do_sample=False)

In [13]:
res[0]

{'summary_text': ' Persib rapuh dihadapan Persebaya Surabaya dalam bentrok big match Liga 1 Rabu (8/12/2021) Persib memainkan kembali formasi 4-3-3 . Maung Bandung tak berdaya di babak kedua, aksi Taisei Marukawa dan Bruno Soares mempermalukan pertahanan Persib lewat tiga gol yang bersarang .'}

In [14]:
' '.join([summ['summary_text'] for summ in res])

' Persib rapuh dihadapan Persebaya Surabaya dalam bentrok big match Liga 1 Rabu (8/12/2021) Persib memainkan kembali formasi 4-3-3 . Maung Bandung tak berdaya di babak kedua, aksi Taisei Marukawa dan Bruno Soares mempermalukan pertahanan Persib lewat tiga gol yang bersarang .  Taisei Marukawa melepas tendangan cerdik terarah membobol gawang M Natshir, 1-0 Persebaya memimpin . Robert langsung merespon gol dengan pergantian pemain, Geoffrey Castillion dimasukkan ganti Febri . Bruno Soares yang mengawali serangan mengakhiri kerjasama satu dua .  Persib hade pisan,  pisan jeblok na,  siga maen teu boga tempo,  kudu jelas ngatur irama da lapang hujan pasti becek . Marselino Ferdinan dengan cerdik memberi trought pass yang gagal diintersept Jupe, Marukawa .'

In [15]:
text = ' '.join([summ['summary_text'] for summ in res])

### Output dalam bentuk txt

In [17]:
with open('Results/summary_text.txt', 'w') as f:
    f.write(text)